In [8]:
import tensorflow as tf
from keras import layers
from keras import preprocessing
import word2vec as w2v
import dataProsessing as dp
import numpy as np
from tensorboard.plugins import projector
import os
import io
import json

Because I have no real idea how to handle filesystems and importing across a branched structure everything is in scripts. I do not apologise.
Furthermore since jupyter notebooks run from their own directory (in this case, Scripts) we run os.chdir("../") to get to the projects root directory (corpus-of-chords). Running this call multiple times without resetting the kernal will continue to crawl up the tree, so do not do that.

In [2]:

#run this once and only once
os.chdir("../")


This loads the dataset and vocab from the data file. To load a different dataset, change the datasetName.

In [12]:
datasetName = "Jsb16thSeparated(t_60+-fifth_rr_re)"

corpus = json.load(open("Data\\Corpi\\" + datasetName + "\\" + datasetName +".json"))
vocab = dataset = json.load(open("Data\\Corpi\\" + datasetName + "\\" + datasetName +"_vocab.json"))
inv_vocab = vocab["inv_vocab"]
vocab = vocab["vocab"]



Here we format the data into valid training samples using keras's built in skipgram generator (which is technically depreciated but don't tell anyone). Note how the model name is derived from hyperparamaters embedding_dim and negative_samples.

In [13]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
embedding_dim = 128
negative_samples = 15
log_dir = "logs\embeddings"
model_name =  "d_" + str(embedding_dim) + " n_" + str(negative_samples)

vectorizedSequences = dp.vectorizeDataset(corpus,vocab)
targets,contexts,labels = [],[],[]
i=0
for s in vectorizedSequences:
    print(i, "/", len(vectorizedSequences))
    i+=1
    tmp = w2v.generateTrainingData(s,vocab_size=len(vocab),window_size=1,negative_samples=negative_samples)
    targets += tmp[0]
    contexts += tmp[1]
    labels += tmp[2]
    
targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

dataset = tf.data.Dataset.from_tensor_slices(((targets,contexts),labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


word2vec = w2v.Word2Vec(len(vocab), embedding_dim,negative_samples)
word2vec.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])




0 / 1146
1 / 1146
2 / 1146
3 / 1146
4 / 1146
5 / 1146
6 / 1146
7 / 1146
8 / 1146
9 / 1146
10 / 1146
11 / 1146
12 / 1146
13 / 1146
14 / 1146
15 / 1146
16 / 1146
17 / 1146
18 / 1146
19 / 1146
20 / 1146
21 / 1146
22 / 1146
23 / 1146
24 / 1146
25 / 1146
26 / 1146
27 / 1146
28 / 1146
29 / 1146
30 / 1146
31 / 1146
32 / 1146
33 / 1146
34 / 1146
35 / 1146
36 / 1146
37 / 1146
38 / 1146
39 / 1146
40 / 1146
41 / 1146
42 / 1146
43 / 1146
44 / 1146
45 / 1146
46 / 1146
47 / 1146
48 / 1146
49 / 1146
50 / 1146
51 / 1146
52 / 1146
53 / 1146
54 / 1146
55 / 1146
56 / 1146
57 / 1146
58 / 1146
59 / 1146
60 / 1146
61 / 1146
62 / 1146
63 / 1146
64 / 1146
65 / 1146
66 / 1146
67 / 1146
68 / 1146
69 / 1146
70 / 1146
71 / 1146
72 / 1146
73 / 1146
74 / 1146
75 / 1146
76 / 1146
77 / 1146
78 / 1146
79 / 1146
80 / 1146
81 / 1146
82 / 1146
83 / 1146
84 / 1146
85 / 1146
86 / 1146
87 / 1146
88 / 1146
89 / 1146
90 / 1146
91 / 1146
92 / 1146
93 / 1146
94 / 1146
95 / 1146
96 / 1146
97 / 1146
98 / 1146
99 / 1146
100 / 1146

Here we have a cell for each file we want to write after the model has fit. This checkpoint is for the tensorboard visualisation.

In [14]:

weights = tf.Variable(word2vec.get_layer('w2v_embedding').get_weights()[0])




checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join("Data\\Corpi\\" + datasetName + "\\tensorboard data\\" + model_name, "embedding.ckpt"))




'data\\corpi\\Jsb16thSeparated(t_60+-fifth_rr_re)\\tensorboard data\\d_128 n_15\\embedding.ckpt-1'

Here we save the weights for use in both generation and the K-means clustering.

In [19]:
weights = np.array(weights).tolist()

with open("Data\\Corpi\\" + datasetName + "\\weights\\" + model_name, "w") as outfile:
    json.dump(weights, outfile)


Here we set up the tensorboard embeddings projector, this is somewhat blackboxed and I have no real idea whats going on. Documentation is scarce and here there be dragons.

In [18]:
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings("Data\\Corpi\\" + datasetName  + "\\tensorboard data\\" + model_name, config)

Here my vs code extention for jupyter lets me run tensorboard. I have no idea why or how but it does. 

In [9]:

%tensorboard --logdir logs\embeddings


UsageError: Line magic function `%tensorboard` not found.
